# TRAINING I3W


# A) Create Envorinment, Vehicles etc

### General Parameter

In [1]:
# Define horizon as a variable to ensure consistent use across notebook (length of one rollout)
HORIZON=500                                 #103 max Horizon, wenn es vor verlassen abbrechen soll!, default war 500

# name of the experiment
experiment_name = "IntersectionExample"

# scenario class
import flow.scenarios as scenarios
print("Available scenarios:")
print(scenarios.__all__)
scenario_name = "IntersectionTWScenario"

# environment class
import flow.multiagent_envs as flowenvs
print("\nAvailable environments:")
print(flowenvs.__all__)
env_name = "MultiAgentIntersectionEnv"

Available scenarios:
['Scenario', 'BayBridgeScenario', 'BayBridgeTollScenario', 'BottleneckScenario', 'Figure8Scenario', 'SimpleGridScenario', 'HighwayScenario', 'LoopScenario', 'MergeScenario', 'TwoLoopsOneMergingScenario', 'MultiLoopScenario', 'IntersectionScenarioTW']

Available environments:
['MultiEnv', 'MultiAgentAccelEnv', 'MultiWaveAttenuationPOEnv', 'MultiAgentIntersectionEnv', 'MultiAgentTeamSpiritIntersectionEnv']


### Net Parameter

In [2]:
from flow.core.params import NetParams
from flow.scenarios.intersection import ADDITIONAL_NET_PARAMS

additionalNetParams = {
            "edge_length": 40,
            "lanes": 1,
            "speed_limit": 30
        }

net_params = NetParams( no_internal_links=False,                  #default: True   !! damit Kreuzungen nicht überspr. werden
                        inflows=None,                             #default: None
                        osm_path=None,                            #default: None
                        netfile=None,                             #default: None
                        additional_params=additionalNetParams     #default: None   !!
                      )

### InitialConfig Parameter

In [3]:
from flow.core.params import InitialConfig

initial_config = InitialConfig( shuffle=True,                            #default: False         !!
                                spacing="custom",                        #default: "uniform"     !!
                                min_gap=10,                              #default: 0
                                perturbation=29.99,                      #default: 0.0            !!        
                                x0=0,                                    #default: 0
                                bunching=0,                              #default: 0
                                lanes_distribution=float("inf"),         #default: float("inf")
                                edges_distribution="all",                #default: "all"
                                additional_params=None )                 #default: None

### SUMO Parameter

In [4]:
from flow.core.params import SumoParams

sumo_params = SumoParams( port = None,                  #default: None
                          sim_step=0.1,                 #default: 0.1
                          emission_path=None,           #default: None
                          lateral_resolution=None,      #default: None
                          no_step_log=True,             #default: True
                          render=False,                 #default: False
                          save_render=False,            #default: False
                          sight_radius=25,              #default: 25
                          show_radius=False,            #default: False
                          pxpm=2,                       #default: 2
                          overtake_right=False,         #default: False    
                          seed=None,                    #default: None
                          restart_instance=False,       #default: False
                          print_warnings=True,          #default: True
                          teleport_time=-1,             #default: -1
                          num_clients=1,                #default: 1
                          sumo_binary=None )            #default: None

### Environment Parameter

In [5]:
from flow.core.params import EnvParams

additionalEnvParams = {
        # maximum acceleration of autonomous vehicles
        "max_accel": 3,
        # maximum deceleration of autonomous vehicles
        "max_decel": 3,
        "target_velocity": 30
    }

env_params = EnvParams( additional_params=additionalEnvParams, #default: None    !!
                        horizon=HORIZON,                       #default: 500     !!
                        warmup_steps=0,                        #default: 0       
                        sims_per_step=1,                       #default: 1
                        evaluate=False )                       #default: False

### Vehicles Parameter

In [6]:
from flow.core.params import VehicleParams

# import vehicles dynamics models
#from flow.controllers import SumoCarFollowingController
from flow.controllers import ContinuousRouter
#from flow.controllers.lane_change_controllers import SumoLaneChangeController
from flow.controllers.lane_change_controllers import StaticLaneChanger
from flow.controllers import RLController
from flow.core.params import SumoLaneChangeParams
from flow.core.params import SumoCarFollowingParams
from random import *

vehicles = VehicleParams()

#### Add RL-Agent controlled vehicles 

In [7]:
# car following parameters, default: None
cf_parameter = SumoCarFollowingParams(
                speed_mode="aggressive")
# lane change parameters, default: None
lc_parameter =  None

vehicles.add( # name of the vehicle
                veh_id = "rl",
              # acceleration controller, default: (SumoCarFollowingController, {})
                acceleration_controller=(RLController, {}),
              # lane_change_controller, default: (SumoLaneChangeController, {})
                lane_change_controller=(StaticLaneChanger,{}),
              # routing controller, default: None
                routing_controller=(ContinuousRouter, {}),
              # initial speed, default: 0
                initial_speed=0,
              # number of vehicles, default: 1 
                num_vehicles=2,
                
                car_following_params=cf_parameter
              # speed mode, default: "right_of_way"
                #speed_mode="aggressive",
              # lane change mode, default: "no_lat_collide"
                #lane_change_mode="aggressive", 
              # car following parameter, default: None
                #sumo_car_following_params=cf_parameter,
              # lane change parameter, default: None
                #sumo_lc_params=lc_parameter
)

### Flow Parameter

In [8]:
# Creating flow_params. Make sure the dictionary keys are as specified. 
flow_params = dict( # name of the experiment
                      exp_tag=experiment_name,
                    # name of the flow environment the experiment is running on
                      env_name=env_name,
                    # name of the scenario class the experiment uses
                      scenario=scenario_name,
                    # simulator that is used by the experiment
                      simulator='traci',
                    # sumo-related parameters (see flow.core.params.SumoParams)
                      sim=sumo_params,
                    # environment related parameters (see flow.core.params.EnvParams)
                      env=env_params,
                    # network-related parameters (see flow.core.params.NetParams and
                    # the scenario's documentation or ADDITIONAL_NET_PARAMS component)
                      net=net_params,
                    # vehicles to be placed in the network at the start of a rollout 
                    # (see flow.core.vehicles.Vehicles)
                      veh=vehicles,
                   # (optional) parameters affecting the positioning of vehicles upon 
                   # initialization/reset (see flow.core.params.InitialConfig)
                      initial=initial_config
                )

# B) Training

In [9]:
import json

import ray
try:
    from ray.rllib.agents.agent import get_agent_class
except ImportError:
    from ray.rllib.agents.registry import get_agent_class
from ray.tune import run_experiments
from ray.tune.registry import register_env

from flow.utils.registry import make_create_env
from flow.utils.rllib import FlowParamsEncoder

from ray import tune
from ray.rllib.agents.ppo.ppo_policy_graph import PPOPolicyGraph

In [10]:
# number of parallel workers
N_CPUS = 2
# number of rollouts per training iteration
N_ROLLOUTS = 20

ray.init(redirect_output=True, num_cpus=N_CPUS+1)

Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-03-14_14-59-27_15055/logs.
Waiting for redis server at 127.0.0.1:41538 to respond...
Waiting for redis server at 127.0.0.1:42848 to respond...
Starting the Plasma object store with 6.554658406 GB memory using /dev/shm.

View the web UI at http://localhost:8889/notebooks/ray_ui.ipynb?token=5529d516fdc919df987ccd1ccf94fefb3de0cf496ff147bb



{'node_ip_address': '172.16.123.117',
 'object_store_addresses': ['/tmp/ray/session_2019-03-14_14-59-27_15055/sockets/plasma_store'],
 'raylet_socket_names': ['/tmp/ray/session_2019-03-14_14-59-27_15055/sockets/raylet'],
 'redis_address': '172.16.123.117:41538',
 'webui_url': 'http://localhost:8889/notebooks/ray_ui.ipynb?token=5529d516fdc919df987ccd1ccf94fefb3de0cf496ff147bb'}

In [11]:
# The algorithm or model to train. This may refer to "
#      "the name of a built-on algorithm (e.g. RLLib's DQN "
#      "or PPO), or a user-defined trainable function or "
#      "class registered in the tune registry.")
alg_run = "PPO"

agent_cls = get_agent_class(alg_run)
config = agent_cls._default_config.copy()
config["num_workers"] = N_CPUS  # number of parallel workers
config["train_batch_size"] = HORIZON * N_ROLLOUTS  # batch size
config["gamma"] = 0.999  # discount rate
config["model"].update({"fcnet_hiddens": [64, 32]})  # size of hidden layers in network
config["use_gae"] = True  # using generalized advantage estimation
config["lambda"] = 0.97  
#config["sgd_minibatch_size"] = min(16 * 1024, config["train_batch_size"])  # stochastic gradient descent
#config["sample_batch_size"] = config["train_batch_size"]/config["num_workers"] # 200 default, trotzdem zu hoch?
config["kl_target"] = 0.02  # target KL divergence
config["num_sgd_iter"] = 10  # number of SGD iterations
config["horizon"] = HORIZON  # rollout horizon

# save the flow params for replay
flow_json = json.dumps(flow_params, cls=FlowParamsEncoder, sort_keys=True,
                       indent=4)  # generating a string version of flow_params
config['env_config']['flow_params'] = flow_json  # adding the flow_params to config dict
config['env_config']['run'] = alg_run

# Call the utility function make_create_env to be able to 
# register the Flow env for this experiment
create_env, gym_name = make_create_env(params=flow_params, version=0)

# Register as rllib env with Gym
register_env(gym_name, create_env)

In [12]:
# multi agent policy mapping
test_env = create_env()
obs_space = test_env.observation_space
act_space = test_env.action_space

def gen_policy():
    return (PPOPolicyGraph, obs_space, act_space, {})

# Setup PG with an ensemble of `num_policies` different policy graphs
policy_graphs = {'rl_0': gen_policy(), 'rl_1': gen_policy()}
    
def policy_mapping_fn(agent_id):
    return agent_id

config.update({
        'multiagent': {
            'policy_graphs': policy_graphs,
            'policy_mapping_fn': tune.function(policy_mapping_fn)
        }
    })

 Starting SUMO on port 44893


11.609282163268665
6.485841348992942


In [ ]:
trials = run_experiments({
    flow_params["exp_tag"]: {
        "run": alg_run,  # RL algorithm to run
        "env": gym_name,  # environment name generated earlier
        "config": {  # configuration params (must match "run" value)
            **config
        },
        "checkpoint_freq": 1,  # number of iterations between checkpoints
        "max_failures": 999,
        "stop": {  # stopping conditions
            "training_iteration": 1000,  # number of iterations to stop after
        },
    },
})

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 0/3 CPUs, 0/1 GPUs
Memory usage on this node: 3.6/16.4 GB

Created LogSyncer for /home/thorsten/ray_results/IntersectionExample/PPO_MultiAgentIntersectionEnv-v0_0_2019-03-14_14-59-309afvmegl -> 
== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 3.7/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv-v0_0:	RUNNING

Result for PPO_MultiAgentIntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-03-14_15-00-51
  done: false
  episode_len_mean: 461.4761904761905
  episode_reward_max: 225.63778774179508
  episode_reward_mean: 50.44007541538655
  episode_reward_min: -152.90116484063756
  episodes_this_iter: 21
  episodes_total: 21
  experiment_id: a07979e1ce864d8ea1fbe22a9b6262d7
  hostname: Gandalf
  info:
    grad_time_ms: 8428.011
    load_time_ms: 205.795
    num_steps_sampled: 10000
    num_s

Result for PPO_MultiAgentIntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-03-14_15-02-42
  done: false
  episode_len_mean: 452.82
  episode_reward_max: 282.4621064690395
  episode_reward_mean: 79.4900714626834
  episode_reward_min: -161.11675332149227
  episodes_this_iter: 23
  episodes_total: 109
  experiment_id: a07979e1ce864d8ea1fbe22a9b6262d7
  hostname: Gandalf
  info:
    grad_time_ms: 4372.321
    load_time_ms: 43.36
    num_steps_sampled: 50000
    num_steps_trained: 50000
    rl_0:
      cur_kl_coeff: 0.012500000186264515
      cur_lr: 4.999999873689376e-05
      entropy: 1.4114465713500977
      kl: 0.003478548489511013
      policy_loss: -0.0024012215435504913
      total_loss: 161.74497985839844
      vf_explained_var: 0.10363620519638062
      vf_loss: 161.74734497070312
    rl_1:
      cur_kl_coeff: 0.012500000186264515
      cur_lr: 4.999999873689376e-05
      entropy: 1.421002745628357
      kl: 0.000883040833286941
      policy_loss: -0.0005220648599788547
     

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 4.7/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv-v0_0:	RUNNING [pid=15100], 258 s, 9 iter, 90000 ts, 73.3 rew

Result for PPO_MultiAgentIntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-03-14_15-04-51
  done: false
  episode_len_mean: 369.99
  episode_reward_max: 352.88021066119705
  episode_reward_mean: 112.82567416225835
  episode_reward_min: -161.6657970811671
  episodes_this_iter: 27
  episodes_total: 244
  experiment_id: a07979e1ce864d8ea1fbe22a9b6262d7
  hostname: Gandalf
  info:
    grad_time_ms: 3870.91
    load_time_ms: 23.153
    num_steps_sampled: 100000
    num_steps_trained: 100000
    rl_0:
      cur_kl_coeff: 0.0003906250058207661
      cur_lr: 4.999999873689376e-05
      entropy: 1.388471245765686
      kl: 0.0014920317335054278
      policy_loss: -0.0014330461854115129
      tota

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 4.7/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv-v0_0:	RUNNING [pid=15100], 377 s, 14 iter, 140000 ts, 162 rew

Result for PPO_MultiAgentIntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-03-14_15-06-49
  done: false
  episode_len_mean: 293.84
  episode_reward_max: 353.29070915225674
  episode_reward_mean: 182.0457125177287
  episode_reward_min: -159.23639323454432
  episodes_this_iter: 36
  episodes_total: 409
  experiment_id: a07979e1ce864d8ea1fbe22a9b6262d7
  hostname: Gandalf
  info:
    grad_time_ms: 3308.827
    load_time_ms: 2.693
    num_steps_sampled: 150000
    num_steps_trained: 150000
    rl_0:
      cur_kl_coeff: 1.220703143189894e-05
      cur_lr: 4.999999873689376e-05
      entropy: 1.3777779340744019
      kl: 0.009372466243803501
      policy_loss: -0.003262387355789542
      tota

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 4.7/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv-v0_0:	RUNNING [pid=15100], 495 s, 19 iter, 190000 ts, 219 rew

Result for PPO_MultiAgentIntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-03-14_15-08-48
  done: false
  episode_len_mean: 230.05
  episode_reward_max: 374.8026812543819
  episode_reward_mean: 215.33033365358585
  episode_reward_min: -159.30313442704693
  episodes_this_iter: 47
  episodes_total: 621
  experiment_id: a07979e1ce864d8ea1fbe22a9b6262d7
  hostname: Gandalf
  info:
    grad_time_ms: 3236.785
    load_time_ms: 2.541
    num_steps_sampled: 200000
    num_steps_trained: 200000
    rl_0:
      cur_kl_coeff: 3.814697322468419e-07
      cur_lr: 4.999999873689376e-05
      entropy: 1.4009674787521362
      kl: 0.0044362470507621765
      policy_loss: -0.0015330089954659343
      to

...

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 5.1/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv-v0_0:	RUNNING [pid=15100], 4653 s, 169 iter, 1690000 ts, 319 rew

Result for PPO_MultiAgentIntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-03-14_16-18-14
  done: false
  episode_len_mean: 132.6
  episode_reward_max: 358.2735176618807
  episode_reward_mean: 321.9342831904561
  episode_reward_min: 275.3589962830865
  episodes_this_iter: 75
  episodes_total: 11869
  experiment_id: a07979e1ce864d8ea1fbe22a9b6262d7
  hostname: Gandalf
  info:
    grad_time_ms: 3381.374
    load_time_ms: 2.457
    num_steps_sampled: 1700000
    num_steps_trained: 1700000
    rl_0:
      cur_kl_coeff: 5.723929920984245e-23
      cur_lr: 4.999999873689376e-05
      entropy: 0.12601986527442932
      kl: 0.020427528768777847
      policy_loss: 0.0011489071184769273
      

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 5.1/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv-v0_0:	RUNNING [pid=15100], 4772 s, 174 iter, 1740000 ts, 321 rew

Result for PPO_MultiAgentIntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-03-14_16-20-13
  done: false
  episode_len_mean: 137.45
  episode_reward_max: 355.1923158888531
  episode_reward_mean: 315.95691711348235
  episode_reward_min: 279.6732908961781
  episodes_this_iter: 73
  episodes_total: 12238
  experiment_id: a07979e1ce864d8ea1fbe22a9b6262d7
  hostname: Gandalf
  info:
    grad_time_ms: 3365.035
    load_time_ms: 2.46
    num_steps_sampled: 1750000
    num_steps_trained: 1750000
    rl_0:
      cur_kl_coeff: 8.58589077939966e-23
      cur_lr: 4.999999873689376e-05
      entropy: 0.2696291506290436
      kl: 0.027404919266700745
      policy_loss: 0.003363653551787138
      to

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 5.1/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv-v0_0:	RUNNING [pid=15100], 4899 s, 179 iter, 1790000 ts, 314 rew

Result for PPO_MultiAgentIntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-03-14_16-22-20
  done: false
  episode_len_mean: 137.51
  episode_reward_max: 351.1968090346899
  episode_reward_mean: 315.7129316022581
  episode_reward_min: 278.1023573176204
  episodes_this_iter: 72
  episodes_total: 12596
  experiment_id: a07979e1ce864d8ea1fbe22a9b6262d7
  hostname: Gandalf
  info:
    grad_time_ms: 3284.572
    load_time_ms: 2.493
    num_steps_sampled: 1800000
    num_steps_trained: 1800000
    rl_0:
      cur_kl_coeff: 4.29294538969983e-23
      cur_lr: 4.999999873689376e-05
      entropy: 0.2589666545391083
      kl: 0.006507233716547489
      policy_loss: 0.001275925082154572
      to

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 5.1/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv-v0_0:	RUNNING [pid=15100], 5018 s, 184 iter, 1840000 ts, 318 rew

Result for PPO_MultiAgentIntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-03-14_16-24-22
  done: false
  episode_len_mean: 140.62
  episode_reward_max: 346.29698737655207
  episode_reward_mean: 315.6784571765328
  episode_reward_min: 279.74752402977816
  episodes_this_iter: 72
  episodes_total: 12954
  experiment_id: a07979e1ce864d8ea1fbe22a9b6262d7
  hostname: Gandalf
  info:
    grad_time_ms: 3533.787
    load_time_ms: 2.583
    num_steps_sampled: 1850000
    num_steps_trained: 1850000
    rl_0:
      cur_kl_coeff: 1.0732363474249576e-23
      cur_lr: 4.999999873689376e-05
      entropy: 0.21886497735977173
      kl: 0.015416436828672886
      policy_loss: -0.001124072354286909
  

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 5.0/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv-v0_0:	RUNNING [pid=15100], 5141 s, 189 iter, 1890000 ts, 313 rew

Result for PPO_MultiAgentIntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-03-14_16-26-23
  done: false
  episode_len_mean: 145.24
  episode_reward_max: 350.1908647660648
  episode_reward_mean: 314.38821084049965
  episode_reward_min: 10.400580736107893
  episodes_this_iter: 72
  episodes_total: 13298
  experiment_id: a07979e1ce864d8ea1fbe22a9b6262d7
  hostname: Gandalf
  info:
    grad_time_ms: 3458.755
    load_time_ms: 2.444
    num_steps_sampled: 1900000
    num_steps_trained: 1900000
    rl_0:
      cur_kl_coeff: 5.366181737124788e-24
      cur_lr: 4.999999873689376e-05
      entropy: 0.15816277265548706
      kl: 0.014654543250799179
      policy_loss: 0.00043008619104512036
  